<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-и-подготовка-данных" data-toc-modified-id="Загрузка-и-подготовка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка и подготовка данных</a></span><ul class="toc-item"><li><span><a href="#Вывод" data-toc-modified-id="Вывод-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Вывод</a></span></li></ul></li><li><span><a href="#Обучение-и-проверка-модели" data-toc-modified-id="Обучение-и-проверка-модели-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение и проверка модели</a></span><ul class="toc-item"><li><span><a href="#Вывод" data-toc-modified-id="Вывод-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Вывод</a></span></li></ul></li><li><span><a href="#Подготовка-к-расчёту-прибыли" data-toc-modified-id="Подготовка-к-расчёту-прибыли-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Подготовка к расчёту прибыли</a></span><ul class="toc-item"><li><span><a href="#Вывод" data-toc-modified-id="Вывод-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Вывод</a></span></li></ul></li><li><span><a href="#Расчёт-прибыли-и-рисков" data-toc-modified-id="Расчёт-прибыли-и-рисков-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Расчёт прибыли и рисков</a></span><ul class="toc-item"><li><span><a href="#Вывод" data-toc-modified-id="Вывод-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Вывод</a></span></li></ul></li><li><span><a href="#Чек-лист-готовности-проекта" data-toc-modified-id="Чек-лист-готовности-проекта-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Чек-лист готовности проекта</a></span></li></ul></div>

# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

<b>Описание данных</b>

Данные геологоразведки трёх регионов находятся в файлах:
* `/datasets/geo_data_0.csv`
* `/datasets/geo_data_1.csv`
* `/datasets/geo_data_2.csv`<br>

`id` — уникальный идентификатор скважины;<br>
`f0`, `f1`, `f2` — три признака точек (неважно, что они означают, но сами признаки значимы);<br>
`product` — объём запасов в скважине (тыс. баррелей).

<b>Условия задачи:</b>
* Для обучения модели подходит только линейная регрессия (остальные — недостаточно предсказуемые).
* При разведке региона исследуют 500 точек, из которых с помощью машинного обучения выбирают 200 лучших для разработки.
* Бюджет на разработку скважин в регионе — 10 млрд рублей.
* При нынешних ценах один баррель сырья приносит 450 рублей дохода. Доход с каждой единицы продукта составляет 450 тыс. рублей, поскольку объём указан в тысячах баррелей.
* После оценки рисков нужно оставить лишь те регионы, в которых вероятность убытков меньше 2.5%. * Среди них выбирают регион с наибольшей средней прибылью.<br>

Данные синтетические: детали контрактов и характеристики месторождений не разглашаются.

## Загрузка и подготовка данных

In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [2]:
try:
    df_1 = pd.read_csv('/datasets/geo_data_0.csv')
    df_2 = pd.read_csv('/datasets/geo_data_1.csv')
    df_3 = pd.read_csv('/datasets/geo_data_2.csv')
except:
    print('При чтение файла возникал ошибка')

Изучим представленные данные

<b>Регион №1

In [3]:
df_1.head()

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


In [4]:
df_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [5]:
df_1.describe()

,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.500419,0.250143,2.502647,92.500000
std,0.871832,0.504433,3.248248,44.288691
min,-1.408605,-0.848218,-12.088328,0.000000
25%,-0.072580,-0.200881,0.287748,56.497507
50%,0.502360,0.250252,2.515969,91.849972
75%,1.073581,0.700646,4.715088,128.564089
max,2.362331,1.343769,16.003790,185.364347


In [6]:
df_1.nunique()

id          99990
f0         100000
f1         100000
f2         100000
product    100000
dtype: int64

Вывод:<br>
Критичных ошибок в данных не обнаружено. В 10 записях имеются повторяющиеся id скважин. 

<b> Регион №2

In [7]:
df_2.head()

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


In [8]:
df_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [9]:
df_2.describe()

,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,1.141296,-4.796579,2.494541,68.825000
std,8.965932,5.119872,1.703572,45.944423
min,-31.609576,-26.358598,-0.018144,0.000000
25%,-6.298551,-8.267985,1.000021,26.953261
50%,1.153055,-4.813172,2.011479,57.085625
75%,8.621015,-1.332816,3.999904,107.813044
max,29.421755,18.734063,5.019721,137.945408


In [10]:
df_2.nunique()

id          99996
f0         100000
f1         100000
f2         100000
product        12
dtype: int64

Вывод:<br>
Значения объемов запасов во втором регионе представлены только 12ью значениями. 4 записи имеют задублированные id скважин.

<b> Регион №3

In [11]:
df_3.head()

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


In [12]:
df_3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [13]:
df_3.describe()

,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.002023,-0.002081,2.495128,95.000000
std,1.732045,1.730417,3.473445,44.749921
min,-8.760004,-7.084020,-11.970335,0.000000
25%,-1.162288,-1.174820,0.130359,59.450441
50%,0.009424,-0.009482,2.484236,94.925613
75%,1.158535,1.163678,4.858794,130.595027
max,7.238262,7.844801,16.739402,190.029838


In [14]:
df_3.nunique()

id          99996
f0         100000
f1         100000
f2         100000
product    100000
dtype: int64

Вывод:<br>
4 записи имеют задублированные id скважин.

Подготовим данные для обучения.

In [15]:
df_1 = df_1.drop(['id'], axis=1)
df_2 = df_2.drop(['id'], axis=1)
df_3 = df_3.drop(['id'], axis=1)

### Вывод

В незначительном количестве записей задублированы id скважин. Объемы запасов во втором регионе представлены только 12 значениями.

## Обучение и проверка модели

Разделим данные на учебные и валидационные.

In [16]:
features_1 = df_1.drop(['product'] , axis=1)
target_1 = df_1['product']

features_2 = df_2.drop(['product'] , axis=1)
target_2 = df_2['product']

features_3 = df_3.drop(['product'] , axis=1)
target_3 = df_3['product']

In [17]:
features_train_1, features_valid_1, target_train_1, target_valid_1 = train_test_split(
    features_1, target_1, test_size=0.25, random_state=12345)

features_train_2, features_valid_2, target_train_2, target_valid_2 = train_test_split(
    features_2, target_2, test_size=0.25, random_state=12345)

features_train_3, features_valid_3, target_train_3, target_valid_3 = train_test_split(
    features_3, target_3, test_size=0.25, random_state=12345)

Проведем обучение.

In [18]:
model_1 = LinearRegression()
model_1.fit(features_train_1,target_train_1)
prediction_1 = model_1.predict(features_valid_1)
rmse_1 = mean_squared_error(target_valid_1, prediction_1)**0.5
mean_target_valid_1 = target_valid_1.mean()
mean_prediction_1 = prediction_1.mean()

model_2 = LinearRegression()
model_2.fit(features_train_2,target_train_2)
prediction_2 = model_2.predict(features_valid_2)
rmse_2 = mean_squared_error(target_valid_2, prediction_2)**0.5
mean_target_valid_2 = target_valid_2.mean()
mean_prediction_2 = prediction_2.mean()

model_3 = LinearRegression()
model_3.fit(features_train_3,target_train_3)
prediction_3 = model_3.predict(features_valid_3)
rmse_3 = mean_squared_error(target_valid_3, prediction_3)**0.5
mean_target_valid_3 = target_valid_3.mean()
mean_prediction_3 = prediction_3.mean()

Выведем результаты.

In [19]:
result_of_learning = pd.DataFrame(index=['средний запас сырья (факт)','средний запас сырья (предсказание)', 'RMSE'], columns=['Регион №1','Регион №2','Регион №3'])
result_of_learning['Регион №1'] = mean_target_valid_1, mean_prediction_1, rmse_1
result_of_learning['Регион №2'] = mean_target_valid_2, mean_prediction_2, rmse_2
result_of_learning['Регион №3'] = mean_target_valid_3, mean_prediction_3, rmse_3
result_of_learning

,Регион №1,Регион №2,Регион №3
средний запас сырья (факт),92.078597,68.723136,94.884233
средний запас сырья (предсказание),92.592568,68.728547,94.965046
RMSE,37.579422,0.893099,40.029709


### Вывод

Средние значения запасов сырья фактические и предсказанные практически совпадают. Низкие (хорошие) оценки RMSE для модели Региона №2 скорее всего обусловленны особенностями данных, представляющих второй регион - все значения запасов в этом регионе представлены только 12тью значениями.

## Подготовка к расчёту прибыли

Для разработки выбирают 200 лучших скважин в регионе.<br>
Бюджет на разработку скважин в регионе — 10 млрд рублей.<br>
Один баррель сырья приносит 450 рублей дохода.<br>
Доход с каждой единицы продукта составляет 450 тыс. рублей, поскольку объём указан в тысячах баррелей.

In [20]:
budget = 10000000
wells_for_production = 200
profit_per_unit = 450

In [21]:
print('Достаточный объём сырья для безубыточной разработки новой скважины: {:.2f} тыс. баррелей'.format(budget / profit_per_unit / wells_for_production))
print('Достаточный объём сырья для безубыточной разработки региона: {:.2f} тыс. баррелей'.format(budget / profit_per_unit))

Достаточный объём сырья для безубыточной разработки новой скважины: 111.11 тыс. баррелей
Достаточный объём сырья для безубыточной разработки региона: 22222.22 тыс. баррелей


### Вывод


Достаточный объём сырья для безубыточной разработки новой скважины должен составлять 111 тыс. баррелей, или 22 222 тыс. баррелей на 200 луших скважин одного региона. При этом как мы ранее выяснили, средний запас одной скважины не привышает 94 тыс. баррелей.

## Расчёт прибыли и рисков 

Сбросим индексы:

In [22]:
target_valid_1 = target_valid_1.reset_index(drop=True)
target_valid_2 = target_valid_2.reset_index(drop=True)
target_valid_3 = target_valid_3.reset_index(drop=True)

prediction_1 = pd.Series(prediction_1)
prediction_2 = pd.Series(prediction_2)
prediction_3 = pd.Series(prediction_3)

Напишем функцию для расчёта прибыли по выбранным скважинам и предсказаниям модели

In [23]:
def revenue(target, prediction, count):
    prediction_sorted = prediction.sort_values(ascending=False)
    selected = target[prediction_sorted.index][:count]
    return profit_per_unit * selected.sum() - budget

In [24]:
print('Прибыль региона № 1 (200 лучших скважин): {:.2f} млрд.рублей'.format(revenue(target_valid_1, prediction_1, 200) / 1000000))
print('Прибыль региона № 2 (200 лучших скважин): {:.2f} млрд.рублей'.format(revenue(target_valid_2, prediction_2, 200) / 1000000))
print('Прибыль региона № 3 (200 лучших скважин): {:.2f} млрд.рублей'.format(revenue(target_valid_3, prediction_3, 200) / 1000000))

Прибыль региона № 1 (200 лучших скважин): 3.32 млрд.рублей
Прибыль региона № 2 (200 лучших скважин): 2.42 млрд.рублей
Прибыль региона № 3 (200 лучших скважин): 2.71 млрд.рублей


Вывод: все 3 региона выгодные, предварительно самый выгодный регион - первый.

Проведем процедуру Bootstrap

In [25]:
state = np.random.RandomState(12345)

def bootstrap_1000(target, prediction):
    
    values = []
    lost = 0
    
    for i in range(1000):
        target_subsample = target.sample(n=500, replace=True, random_state=state)
        prediction_subsample = prediction[target_subsample.index] 
        profit= revenue(target_subsample, prediction_subsample, 200)
        if profit < 0:
            lost +=1
        values.append(profit)

    values = pd.Series(values)   
    lower = values.quantile(0.025)
    upper = values.quantile(0.975)
    average = values.mean()
    risk = lost / 1000
    
    print('Средняя прибыль по региону = {:.2f} млн.руб.'.format(average / 1000))
    print('95% доверительный интервал от {:.2f} до {:.2f} млн.руб.'.format(lower / 1000, upper / 1000))
    print('Процент риска {:.2%}'.format(risk))

<b>Регион №1

In [26]:
bootstrap_1000(target_valid_1, prediction_1)

Средняя прибыль по региону = 425.94 млн.руб.
95% доверительный интервал от -102.09 до 947.98 млн.руб.
Процент риска 6.00%


<b>Регион №2

In [27]:
bootstrap_1000(target_valid_2, prediction_2)

Средняя прибыль по региону = 518.26 млн.руб.
95% доверительный интервал от 128.12 до 953.61 млн.руб.
Процент риска 0.30%


<b>Регион №3

In [28]:
bootstrap_1000(target_valid_3, prediction_3)

Средняя прибыль по региону = 420.19 млн.руб.
95% доверительный интервал от -115.85 до 989.63 млн.руб.
Процент риска 6.20%


### Вывод

 На основе проведенного анализа наиболее предпочительным оказался второй регион, т.к. он обладает самым низким процентом риска и наибольшей средней прибылью по сравнению с двумя другими регионами.